In [ ]:
!pip install pandas

In [1]:
from dotenv import load_dotenv
load_dotenv()

from langchain_community.chat_models import ChatOpenAI
llm = ChatOpenAI()


/home/ninoue/trial/venv/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:189: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use langchain_openai.ChatOpenAI instead.
  warn_deprecated(


In [2]:
from langchain import hub
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.chat_models import ChatOpenAI
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.embeddings import OpenAIEmbeddings
from langchain_community.vectorstores import Chroma
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

In [10]:
from langchain_community.document_loaders.csv_loader import CSVLoader

loader = CSVLoader(
    file_path="./emails.csv", 
    encoding="utf-8",
    csv_args={
        "delimiter": ",",
        "quotechar": '"',
        "fieldnames": ["From", "To", "Date", "Subject", "Message-Id", "In-Reply-To", "References", "Body"]
    },
)
data = loader.load()



In [12]:
print(data[:3])


[Document(page_content='From: From\nTo: To\nDate: Date\nSubject: Subject\nMessage-Id: Message-Id\nIn-Reply-To: In-Reply-To\nReferences: References\nBody: Body', metadata={'source': './emails.csv', 'row': 0}), Document(page_content='From: Naoya Inoue <ninoue@galapagos-island.net>\nTo: 井上直也 <inoue405@gmail.com>\nDate: Fri, 05 Jan 2024 13:13:45 +0000\nSubject: Re: icloudの容量を節約したい。\nMessage-Id: <a586cab4-1906-41d9-a1fe-f228ba3820e7@me.com>\nIn-Reply-To: <CAE_FsL=VE959YOAJ3_o3Uo4gF6viXj2b_67hv3i8LbnS1ZG1Zw@mail.gmail.com>\nReferences: <CAE_FsLk_to1+gMDipA4UqcnK1nPKzuwum0i8oue7s9N-HSx4yQ@mail.gmail.com> <daf3b831-c984-49cf-8bdc-3c62da0ef0ba@me.com> <CAE_FsLnQSXbP0y+x+BmsHphjPSe=dtWX9tC61dZMiAcqgakO3Q@mail.gmail.com> <ecb8b1cb-1f6a-4408-9711-ff802919ebac@me.com> <CAE_FsL=VE959YOAJ3_o3Uo4gF6viXj2b_67hv3i8LbnS1ZG1Zw@mail.gmail.com>\nBody: Fire TVって一つあれば大丈夫ですよ。 2024年1月5日 22:13、井上直也 <inoue405@gmail.com>からのメール： ありがとうございます。Amazon photoをインストールして試して見ます。 写真データの出し入れが自在に出来そうであれば、もう一つfire TVを購入して使うか、なんだか

In [20]:
len(data[0].page_content)
print(data[2].page_content)

From: Naoya Inoue <ninoue@galapagos-island.net>
To: 井上直也 <inoue405@gmail.com>
Date: Fri, 05 Jan 2024 13:11:12 +0000
Subject: Re: icloudの容量を節約したい。
Message-Id: <daf3b831-c984-49cf-8bdc-3c62da0ef0ba@me.com>
In-Reply-To: <CAE_FsLk_to1+gMDipA4UqcnK1nPKzuwum0i8oue7s9N-HSx4yQ@mail.gmail.com>
References: <CAE_FsLk_to1+gMDipA4UqcnK1nPKzuwum0i8oue7s9N-HSx4yQ@mail.gmail.com>
Body: Apple４K TVを、使うことが前提ですかね。 それならiCloud写真一択だと思うのですよ。 iPhoneやiPadでミラーリングさせれば良いので、想定している使い方は可能です。 AppleTV用にAmazonPhotosアプリがあれば、AppleTVで直接写真閲覧もできるはずですが、アプリがあるかは未確認です。 その他、FireTVを導入する手もありますね。 2024年1月5日 22:10、井上直也 <inoue405@gmail.com>からのメール： GoogleやOneDriveなどのApple以外のクラウドサービスを利用する手はありますが、量が多いと費用が発生します。 また、写真アプリではアクセスできず、ファイルアプリや専用アプリからのアクセスになります。 ちなみに、AmazonのPrime会員なら、無制限で写真を保存できます。 Amazon Photos 動画には上限ありますが、写真は無制限でフル解像度で保存できるので、古い写真を退避させるには良いかもです。 写真の管理や閲覧には専用アプリが必要です。 2024年1月5日(金) 22:08 Naoya Inoue < ninoue@galapagos-island.net >: なるほど、Icloudの仕組みがよくわからなくですみません。 Apple TVやPCにicloudの容量を超えるオリジナルを保存して使うことはできないんですね。 御指南いただいた方法でやってみま

In [ ]:
print(len(data[6].page_content))

In [42]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=800, chunk_overlap=200, add_start_index=True)
splits = text_splitter.split_documents(data)

In [35]:
print(splits)
len(splits[0].page_content)

[Document(page_content='From: From\nTo: To\nDate: Date\nSubject: Subject\nMessage-Id: Message-Id\nIn-Reply-To: In-Reply-To\nReferences: References\nBody: Body', metadata={'source': './emails.csv', 'row': 0, 'start_index': 0}), Document(page_content='From: Naoya Inoue <ninoue@galapagos-island.net>\nTo: 井上直也 <inoue405@gmail.com>\nDate: Fri, 05 Jan 2024 13:13:45 +0000\nSubject: Re: icloudの容量を節約したい。\nMessage-Id: <a586cab4-1906-41d9-a1fe-f228ba3820e7@me.com>\nIn-Reply-To: <CAE_FsL=VE959YOAJ3_o3Uo4gF6viXj2b_67hv3i8LbnS1ZG1Zw@mail.gmail.com>\nReferences: <CAE_FsLk_to1+gMDipA4UqcnK1nPKzuwum0i8oue7s9N-HSx4yQ@mail.gmail.com> <daf3b831-c984-49cf-8bdc-3c62da0ef0ba@me.com> <CAE_FsLnQSXbP0y+x+BmsHphjPSe=dtWX9tC61dZMiAcqgakO3Q@mail.gmail.com> <ecb8b1cb-1f6a-4408-9711-ff802919ebac@me.com> <CAE_FsL=VE959YOAJ3_o3Uo4gF6viXj2b_67hv3i8LbnS1ZG1Zw@mail.gmail.com>', metadata={'source': './emails.csv', 'row': 1, 'start_index': 0}), Document(page_content='Body: Fire TVって一つあれば大丈夫ですよ。 2024年1月5日 22:13、井上直也 <inoue4

127

In [43]:
vectorstore = Chroma.from_documents(documents=splits, embedding=OpenAIEmbeddings())

/home/ninoue/trial/venv/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:189: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use langchain_openai.OpenAIEmbeddings instead.
  warn_deprecated(


In [37]:
retriever = vectorstore.as_retriever(search_type="similarity", search_kwargs={"k": 6})
retrieved_docs = retriever.invoke("やりたいことは、何ですか？")

print(retrieved_docs[0].page_content)

From: Naoya Inoue <ninoue@galapagos-island.net>
To: 井上直也 <inoue405@gmail.com>
Date: Fri, 05 Jan 2024 13:04:25 +0000
Subject: Re: icloudの容量を節約したい。
Message-Id: <0de6a730-8329-446c-9198-bc62e1e56be8@me.com>
In-Reply-To: <CAE_FsLmB+4i6PVryHcEORiHnjkot0kGhzYEoZ7X138YLgTRbSQ@mail.gmail.com>
References: <CAE_FsLmB+4i6PVryHcEORiHnjkot0kGhzYEoZ7X138YLgTRbSQ@mail.gmail.com>
Body: 一度googleフォトに入れたことがあったのですが、仕組みがわかりませんがgoogleと同期すると、Appleフォトにも影響があり、オリジナルがGoogleに移ってしまい、容量がいっぱいになったら料金請求されたので、元に戻そうとしたら、データが元通りにAppleに戻らなくなってしまい、色々調べて苦労して元に戻した記憶があります。それ以来、他のクラウドには手を出さないようにしています。 やっぱり、大事なデータを保管するためにはそれなりの対価を支払う必要があるんですね。 今は、IOデータのHDDにバックアップしていますが、IOデータもアップルのバックアップアプリのサポートを辞めてしまったのでいつまで使えるかわかりません。 長くなりましたが、ありがとうございました。また色々教えてください。 2024年1月5日 22:02、井上直也 <inoue405@gmail.com>からのメール： 写真が３万枚を超えてきてiClouｄの９０パーセントになっています。icloudの量を節約するために写真データの一部をHDDやPCに保存して見たい時に写真アプリを使ってみることはできますか？ icloud写真を使う場合、オリジナルがicloudの容量を超える場合はお金を払って容量を増やすしかないのでしょうでしょうか？ 現在１００Gです。今以上に毎月の使用量を払いたくないのです。 よろしくお願いします。


In [40]:
print(retrieved_docs[3].page_content)

Body: なるほど、Icloudの仕組みがよくわからなくですみません。 Apple TVやPCにicloudの容量を超えるオリジナルを保存して使うことはできないんですね。 御指南いただいた方法でやってみます。 ありがとうございました。 Apple 4K TV使用前提ではありませんが、やりたいことは今まで貯めてきた写真を管理して、大きな画面でみんなで楽しみたいのです。 iphoneだけでは出来ない様なので安いミラーリングのコネクターを購入しましたがうまくいきませんでした。 iphoneユーザーで大量の写真を皆さんどのように管理されているのでかなと思いました。 やっぱりicloudで容量追加するのが一番の様ですね。自宅のHDDに保存できるといいなと思いましたが・・・ 2024年1月5日 22:07、井上直也 <inoue405@gmail.com>からのメール： >HDDやPCに保存して見たい時に写真アプリを使ってみることはできますか？ 写真アプリに外部のリソースを参照する機能はないので、自宅のPC（HDD）をネット上に公開しても不可能じゃないですかね。 写真アプリを使いたいなら、一旦PC（HDD）に古い写真を退避して、共有アルバムに見たい写真だけアップすれば良いのでは？ 5000枚が上限ですが、iCloudの容量を圧迫しませんので。 iPhone、iPad、Mac の写真アプリで共有アルバムを使う方法 - Apple サポート (日本) 2024年1月5日(金) 22:02 井上直也 < inoue405@gmail.com >: 写真が３万枚を超えてきてiClouｄの９０パーセントになっています。icloudの量を節約するために写真データの一部をHDDやPCに保存して見たい時に写真アプリを使ってみることはできますか？ icloud写真を使う場合、オリジナルがicloudの容量を超える場合はお金を払って容量を増やすしかないのでしょうでしょうか？ 現在１００Gです。今以上に毎月の使用量を払いたくないのです。 よろしくお願いします。


In [44]:
llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0)
prompt = hub.pull("rlm/rag-prompt")

def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

from langchain_core.runnables import RunnableParallel
rag_chain_from_docs = (
    RunnablePassthrough.assign(context=(lambda x: format_docs(x["context"])))
    | prompt
    | llm
    | StrOutputParser()
)

rag_chain_with_source = RunnableParallel(
    {"context": retriever, "question": RunnablePassthrough()}
).assign(answer=rag_chain_from_docs)

rag_chain_with_source.invoke("やりたいことは、何ですか？")



/home/ninoue/trial/venv/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:189: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use langchain_openai.ChatOpenAI instead.
  warn_deprecated(
/home/ninoue/trial/venv/lib/python3.10/site-packages/langchain_core/_api/beta_decorator.py:160: LangChainBetaWarning: The function `loads` is in beta. It is actively being worked on, so the API may change.
  warn_beta(


{'context': [Document(page_content='Inoue < ninoue@galapagos-island.net >: Apple４K TVを、使うことが前提ですかね。 それならiCloud写真一択だと思うのですよ。 iPhoneやiPadでミラーリングさせれば良いので、想定している使い方は可能です。 AppleTV用にAmazonPhotosアプリがあれば、AppleTVで直接写真閲覧もできるはずですが、アプリがあるかは未確認です。 その他、FireTVを導入する手もありますね。 2024年1月5日 22:10、井上直也 < inoue405@gmail.com >からのメール： GoogleやOneDriveなどのApple以外のクラウドサービスを利用する手はありますが、量が多いと費用が発生します。 また、写真アプリではアクセスできず、ファイルアプリや専用アプリからのアクセスになります。 ちなみに、AmazonのPrime会員なら、無制限で写真を保存できます。 Amazon Photos 動画には上限ありますが、写真は無制限でフル解像度で保存できるので、古い写真を退避させるには良いかもです。 写真の管理や閲覧には専用アプリが必要です。 2024年1月5日(金) 22:08 Naoya Inoue < ninoue@galapagos-island.net >: なるほど、Icloudの仕組みがよくわからなくですみません。 Apple TVやPCにicloudの容量を超えるオリジナルを保存して使うことはできないんですね。 御指南いただいた方法でやってみます。 ありがとうございました。 Apple 4K TV使用前提ではありませんが、やりたいことは今まで貯めてきた写真を管理して、大きな画面でみんなで楽しみたいのです。', metadata={'row': 1, 'source': './emails.csv', 'start_index': 1188}),
  Document(page_content='22:02 井上直也 < inoue405@gmail.com >: 写真が３万枚を超えてきてiClouｄの９０パーセントになっています。icloudの量を節約するために写真データの一部をHDDやPCに保存して見たい時に写真アプリを使